In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
import glob 
from tqdm.notebook import tqdm
from torch.utils.data import Dataset, DataLoader
import random
import os
from PIL import Image
import torch
from torchvision import transforms
import torchvision.models as models
import torch.nn as nn
import os
from datetime import datetime
import time

In [2]:
# basic parameter
grade2class={'A':0, 'B':1, 'C':2}
FOLD_NUMBER = 0 # the validation fold index
NUMBER_OF_CLASSES = 3
TRAIN_ROOT_PATH = '../data/preprocess/all'

In [3]:
# hyper parameter
class TrainGlobalConfig:
    # basic setting
    batch_size = 12
    n_epochs = 40 
    lr = 0.001

    folder = 'model_bins/EfficientNet_B0'   # path to save trained model
    
    # wether to print out information
    verbose = True          
    verbose_step = 1       
    
    # scheduler parameter
    step_scheduler = False  # do scheduler.step after optimizer.step
    validation_scheduler = True  # do scheduler.step after validation stage loss
    
    SchedulerClass = torch.optim.lr_scheduler.ReduceLROnPlateau
    scheduler_params = dict(
        mode='min',             # The being monitored matric should going down(min)/up(max)
        factor=0.1,             # new_lr = factor * lr
        patience=1,             # If the monitored matric do not goes down/up for continuse "patient" epochs
                                # update the learning rate
        verbose=False,          # Wether the print out message
        threshold=0.0001,       # Threshold for measuring the new optimum, to only focus on significant changes
        threshold_mode='abs',   # Inprovement = best - threshold
        cooldown=0,             # After turn down the lr, freeze lr for "cooldown" epoch
        min_lr=1e-7,
        eps=1e-08               # The minimin value that learning rate decay
    )

In [4]:
# read csv file
all_df = pd.read_csv('/home/zxcvbn7222/machine_learning/芒果2.0/data/preprocess/all.csv')

In [5]:
print(all_df.shape)
all_df.head()

(52000, 9)


,Unnamed: 0,image_id,grade,pos_x,pos_y,width,height,area,square ratio
0,0,31191.jpg,A,345,95,516,530,273480,1.027132
1,1,77679.jpg,A,310,73,569,520,295880,1.094231
2,2,71334.jpg,B,215,153,507,504,255528,1.005952
3,3,17095.jpg,A,265,76,598,576,344448,1.038194
4,4,48491.jpg,A,281,148,525,554,290850,1.055238


In [6]:
# for 5-Folds cross-validato
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [7]:
# assign fold index to all sample
all_df.loc[:, 'fold'] = 0
for fold_number, (train_index, val_index) in enumerate(skf.split(X=all_df.image_id, y=all_df.grade)):
    all_df.loc[all_df.iloc[val_index].index, 'fold'] = fold_number

In [8]:
all_df.head()

,Unnamed: 0,image_id,grade,pos_x,pos_y,width,height,area,square ratio,fold
0,0,31191.jpg,A,345,95,516,530,273480,1.027132,3
1,1,77679.jpg,A,310,73,569,520,295880,1.094231,4
2,2,71334.jpg,B,215,153,507,504,255528,1.005952,3
3,3,17095.jpg,A,265,76,598,576,344448,1.038194,2
4,4,48491.jpg,A,281,148,525,554,290850,1.055238,0


In [9]:
# Data augmentation
def get_train_transforms(resize_ratio):
    base_resolution = 200
    resize_h = int(base_resolution * resize_ratio)
    resize_w = int(resize_h * 1.21)
     
    return transforms.Compose([
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomVerticalFlip(p=0.5),
        #transforms.Resize((resize_h, resize_w)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

def get_valid_transforms():
    return transforms.Compose([
        transforms.Resize((224, int(224*1.21))),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

In [10]:
class DatasetRetriever(Dataset):

    def __init__(self, df, grade2class,train):
        super().__init__()
        
        self.df = df                       # the processed csv file    
        self.train = train
        self.grade2class = grade2class
        # self.new_transform()
        if train: self.new_transform()
        else: self.transforms = get_valid_transforms()
        
    # call this function to reset the size of training image
    def new_transform(self):               
        if self.train: self.transforms = get_train_transforms(random.uniform(1,2))

    def __getitem__(self, index: int):
        # load image
        sample = self.df.iloc[index]
        path = os.path.join(TRAIN_ROOT_PATH, sample['image_id'])
        with open(path, 'rb') as f:
            img = Image.open(f)
            img.convert('RGB')
        
        target = torch.tensor(grade2class[sample['grade']], dtype=torch.int64)
        img = self.transforms(img)
        return img, target
    
    def __len__(self) -> int:
        return self.df.shape[0]

In [11]:
'''
train_data = DatasetRetriever(all_df, grade2class, True)
loader = DataLoader(train_data,batch_size=10)

for data in loader:
    loader.dataset.new_transform()
    #display(trans(data[0][0]))
    print(data[0].shape, data[1].shape)
'''

'\ntrain_data = DatasetRetriever(all_df, grade2class, True)\nloader = DataLoader(train_data,batch_size=10)\n\nfor data in loader:\n    loader.dataset.new_transform()\n    #display(trans(data[0][0]))\n    print(data[0].shape, data[1].shape)\n'

In [12]:
# instance DatasetRetriever
train_data = DatasetRetriever(all_df[all_df['fold']!=FOLD_NUMBER], grade2class, train=True)
val_data = DatasetRetriever(all_df[all_df['fold']==FOLD_NUMBER], grade2class, train=False)

In [13]:
img, tar = train_data[0]
print(img.shape)
print(tar.shape)

torch.Size([3, 307, 371])
torch.Size([])


In [14]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.correct_count = 0
        self.count = 0
        self.acc = 0

    def update(self, val, batch_size, ncorrect):
        self.val = val
        self.sum += val * batch_size
        self.count += batch_size
        self.correct_count += ncorrect
        self.avg = self.sum / self.count

In [15]:
def rightness(predictions, labels):
    
    pred = torch.max(predictions.data, 1)[1]   # the result of prediction
    rights = pred.eq(labels.data.view_as(pred)).sum()  # count number of correct example
    
    return rights

In [16]:
class Fitter:
    def __init__(self, model, device, config):
        # assign parameter
        self.model = model
        self.device = device
        self.config = config
        
        self.epoch = config.n_epochs
        
        # the folder saving the trained model, if the folder is not exist, create it
        self.base_dir = f'../{config.folder}'
        if not os.path.exists(self.base_dir):
            os.makedirs(self.base_dir)
        
        self.log_path = f'{self.base_dir}/log.txt'
        self.best_loss = 10**5   
        
        
        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = torch.optim.SGD(self.model.parameters(), lr=config.lr)
        '''
        torch.optim.lr_scheduler.ReduceLROnPlateau allows dynamic learning rate reducing based on 
        some validation measurements
        '''
        self.scheduler = config.SchedulerClass(self.optimizer, **config.scheduler_params)
        
        
        self.log(f'Fitter prepared. Device is {self.device}') 

    def fit(self, train_loader, validation_loader):
        for e in range(self.config.n_epochs):
            if self.config.verbose:
                lr = self.optimizer.param_groups[0]['lr']
                timestamp = datetime.utcnow().isoformat()
                self.log(f'\n{timestamp}\nLR: {lr}')
            
            # train
            t = time.time()
            loss = self.train_one_epoch(train_loader)
            self.log(f'[RESULT]: Train. Epoch: {self.epoch}, loss: {loss.avg:.5f}, acc: {loss.acc:.5f},time: {(time.time() - t):.5f}')
            self.save(f'{self.base_dir}/last-checkpoint.bin') 
            
            # validation
            t = time.time()
            loss = self.validation(validation_loader)
            self.log(f'[RESULT]: Val. Epoch: {self.epoch}, loss: {loss.avg:.5f}, acc: {loss.acc:.5f},time: {(time.time() - t):.5f}')
            
            # if the new model is better, save it, otherwise abandant
            if loss.avg < self.best_loss:
                self.best_loss = loss.avg
                self.model.eval()
                self.save(f'{self.base_dir}/best-checkpoint-{str(self.epoch).zfill(3)}epoch.bin')
            
            # update learning rate
            if self.config.validation_scheduler:self.scheduler.step(metrics=loss.avg)
            self.epoch += 1

    def validation(self, val_loader):
        self.model.eval()
        
        # record information
        loss_recorder = AverageMeter()   
        t = time.time()
        
        for step, (data, target) in enumerate(val_loader):
            if self.config.verbose:
                if step % self.config.verbose_step == 0:
                    print(
                        f'Val Step {step}/{len(val_loader)}, ' + \
                        f'loss: {loss_recorder.avg:.5f}, ' + \
                        f'time: {(time.time() - t):.5f}', end='\r'
                    )
            with torch.no_grad():
                batch_size = data.shape[0]
                data, target = data.cuda(), target.cuda()

                output = self.model(data)
                loss = self.criterion(output, target)

            loss_recorder.update(loss.detach().item(), batch_size, int(rightness(output, target).cpu()))

            
        loss_recorder.acc = loss_recorder.correct_count / loss_recorder.count
        return loss_recorder

    def train_one_epoch(self, train_loader):
        self.model.train()
        
        loss_recorder = AverageMeter()
        t = time.time()
        
        pbar = tqdm(total=len(train_loader))
        for step, (data, target) in enumerate(train_loader):
            train_loader.dataset.new_transform()
            if self.config.verbose:
                if step % self.config.verbose_step == 0:
                    print(
                        f'Train Step {step}/{len(train_loader)}, ' + \
                        f'loss: {loss_recorder.avg:.5f}, ' + \
                        f'time: {(time.time() - t):.5f}', end='\r'
                    )
            
            batch_size = data.shape[0]
            data, target = data.cuda(), target.cuda()
            output = self.model(data)
            loss = self.criterion(output, target)
            
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
            if self.config.step_scheduler: self.scheduler.step()
            loss_recorder.update(loss.detach().item(), batch_size, int(rightness(output, target).cpu()))
            pbar.update(1)

        loss_recorder.acc = loss_recorder.correct_count / loss_recorder.count
        return loss_recorder
    
    def save(self, path):
        self.model.eval()
        torch.save({
            'model_state_dict': self.model.state_dict(),
            'optimizer_state_dict': self.optimizer.state_dict(),
            'scheduler_state_dict': self.scheduler.state_dict(),
            'best_summary_loss': self.best_loss,
            'epoch': self.epoch,
        }, path)

    def load(self, path):
        checkpoint = torch.load(path)
        self.model.model.load_state_dict(checkpoint['model_state_dict'])
        self.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        self.scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
        self.best_summary_loss = checkpoint['best_loss']
        self.epoch = checkpoint['epoch'] + 1
    
    # print and write information in log
    def log(self, message):
        if self.config.verbose:print(message)
        with open(self.log_path, 'a+') as logger:
            logger.write(f'{message}\n')

In [17]:
def run_training(net):
    device = torch.device('cuda:0')
    net.to(device)

    train_loader = torch.utils.data.DataLoader(
        train_data,
        batch_size=TrainGlobalConfig.batch_size,
        shuffle = True,
        drop_last = False  # if the remaining sample is less than a batch, discard it
    )
    
    val_loader = torch.utils.data.DataLoader(
        val_data, 
        batch_size=TrainGlobalConfig.batch_size,
        num_workers=8,
        shuffle=False
    )

    fitter = Fitter(model=net, device=device, config=TrainGlobalConfig)
    fitter.fit(train_loader, val_loader)

In [18]:
from efficientnet_pytorch import EfficientNet
model = EfficientNet.from_pretrained('efficientnet-b0')

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b0-355c32eb.pth" to /home/zxcvbn7222/.cache/torch/hub/checkpoints/efficientnet-b0-355c32eb.pth



Loaded pretrained weights for efficientnet-b0


In [20]:
print(model._fc)

Linear(in_features=1280, out_features=1000, bias=True)


In [19]:
#model = models.resnet18(pretrained=True)
#model.fc = nn.Linear(model.fc.in_features, NUMBER_OF_CLASSES)

In [20]:
run_training(model)

Fitter prepared. Device is cuda:0

2020-08-02T11:23:02.245793
LR: 0.001


[RESULT]: Train. Epoch: 40, loss: 1.57089, acc: 0.63240,time: 1416.31103
[RESULT]: Val. Epoch: 40, loss: 1.15523, acc: 0.68385,time: 39.08877

2020-08-02T11:47:17.787918
LR: 0.001


[RESULT]: Train. Epoch: 41, loss: 0.76745, acc: 0.69748,time: 1419.43647
[RESULT]: Val. Epoch: 41, loss: 0.86318, acc: 0.71760,time: 40.18635

2020-08-02T12:11:37.585651
LR: 0.001


[RESULT]: Train. Epoch: 42, loss: 0.71848, acc: 0.71373,time: 1373.66419
[RESULT]: Val. Epoch: 42, loss: 0.75680, acc: 0.73413,time: 39.19434

2020-08-02T12:35:10.548120
LR: 0.001


[RESULT]: Train. Epoch: 43, loss: 0.68279, acc: 0.72623,time: 1356.56615
[RESULT]: Val. Epoch: 43, loss: 0.77079, acc: 0.74279,time: 39.25549

2020-08-02T12:58:26.437717
LR: 0.001


...

[RESULT]: Train. Epoch: 68, loss: 0.53756, acc: 0.78550,time: 1186.44622
[RESULT]: Val. Epoch: 68, loss: 0.56500, acc: 0.78596,time: 38.49552

2020-08-02T21:51:04.232992
LR: 1.0000000000000002e-07


[RESULT]: Train. Epoch: 69, loss: 0.53858, acc: 0.78493,time: 1191.17237
[RESULT]: Val. Epoch: 69, loss: 0.54283, acc: 0.78904,time: 38.46444

2020-08-02T22:11:33.970110
LR: 1.0000000000000002e-07


[RESULT]: Train. Epoch: 70, loss: 0.53842, acc: 0.78596,time: 1203.86922
[RESULT]: Val. Epoch: 70, loss: 0.56322, acc: 0.78442,time: 38.48565

2020-08-02T22:32:16.399838
LR: 1.0000000000000002e-07


[RESULT]: Train. Epoch: 71, loss: 0.54028, acc: 0.78425,time: 1197.55583
[RESULT]: Val. Epoch: 71, loss: 0.55056, acc: 0.78769,time: 38.46510

2020-08-02T22:52:52.495769
LR: 1.0000000000000002e-07


KeyboardInterrupt: 